<a href="https://colab.research.google.com/github/OrJaku/dm_sign_recognise/blob/master/Day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import os
import datetime

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin


In [0]:
pip install hyperopt

In [6]:
cd drive/My Drive/Colab Notebooks/dw_matrix_3/dm_sign_recognise

/content/drive/My Drive/Colab Notebooks/dw_matrix_3/dm_sign_recognise


# Dawnload and preapare data

In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

x_train, y_train = train['features'], train['labels']
x_test, y_test = test['features'], test['labels']

if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = x_train.shape[1:]
num_classes = y_train.shape[1]


df = pd.read_csv('data/signnames.csv')
lables_dict = df.to_dict()['b']



# Model compile and fit

In [0]:
def train_model(model, x_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir=os.path.join('logs', str(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      x_train, 
      y_train,
      batch_size=params_fit.get('barch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (x_train, y_train)),
      callbacks= [tensorboard_callback]


      )
  return model

# Predict function

In [0]:
def predict(model_trained, x_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(x_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)
  

# Creating CNN 


In [0]:
def get_cnn(input_shape, num_classes):
  return  Sequential([
            Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
            Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding="same"),
            MaxPool2D(),
            Dropout(0.3),

            Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"),
            Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
            MaxPool2D(),
            Dropout(0.3),

            
            Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"),
            Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
            MaxPool2D(),
            Dropout(0.3),

            Flatten(),

            Dense(1024, activation='relu'),
            Dropout(0.3),

            Dense(1024, activation='relu'),
            Dropout(0.3),
            
            Dense(num_classes, activation='softmax')
    
  ])

## Converting *RGB* image to *Gray*

In [0]:
x_train_gray = color.rgb2gray( x_train ).reshape(-1, 32, 32, 1)
x_test_gray = color.rgb2gray( x_test ).reshape(-1, 32, 32, 1)

# Run model 

In [0]:
params_fit= {
    'epochs':8
}


model = get_cnn_v6((32, 32, 1), num_classes)
model_trained = train_model(model, x_train_gray, y_train, params_fit)
predict(model_trained, x_test_gray, y_test)


--------- Optimalization ---------

In [0]:
def get_model(params):
  return  Sequential([
            Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
            Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding="same"),
            MaxPool2D(),
            Dropout(params['dropout_cnn_block_1']),

            Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding="same"),
            Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
            MaxPool2D(),
            Dropout(params['dropout_cnn_block_2']),

            
            Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding="same"),
            Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
            MaxPool2D(),
            Dropout(params['dropout_cnn_block_3']),

            Flatten(),

            Dense(1024, activation='relu'),
            Dropout(params['dropout_dense_block_1']),

            Dense(1024, activation='relu'),
            Dropout(params['dropout_dense_block_2']),
            
            Dense(num_classes, activation='softmax')
    
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      x_train, 
      y_train,
      batch_size=int(params.get('barch_size', 128)),
      epochs=8,
      verbose=0,
      )
  score = model.evaluate(x_test, y_test)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [0]:
space = {
    'barch_size': hp.quniform('barch_size', 100,200, 10),
    'dropout_cnn_block_1': hp.uniform('dropout_cnn_block_1', 0.3, 0.5),
    'dropout_cnn_block_2': hp.uniform('dropout_cnn_block_2', 0.3, 0.5),
    'dropout_cnn_block_3': hp.uniform('dropout_cnn_block_3', 0.3, 0.5),
    'dropout_dense_block_1': hp.uniform('dropout_dense_block_1', 0.3, 0.5),
    'dropout_dense_block_2': hp.uniform('dropout_dense_block_2', 0.3, 0.5),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)


{'barch_size': 130.0, 
'dropout_cnn_block_1': 0.4179350709693718, 
'dropout_cnn_block_2': 0.39684638781089915, 
'dropout_cnn_block_3': 0.38237954774632404, 
'dropout_dense_block_1': 0.367459765552649, 
'dropout_dense_block_2': 0.4238439232584229}
accuracy=0.9743764400482178